# Multilabel Classificator
The results obtained with the CNN models for just one label classification where is not great enough, maybe de cause of those results is that some songs are not only classify as one genre, it could be in various. Due to this  possible reason, this notebook has been created.

## Initialisation

### Libraries

In [1]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.optimizers import SGD, Adam
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import numpy as np
import os
import ast
from pathlib import Path

# Import function to plot the results
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import plots

2024-01-14 20:22:33.932742: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Data Configuration Parameters
Configuration variables related to the data

In [2]:
# Randomize the initial network weights
random_seed = True

# Paths to where training, testing, and validation images are
database_dir = 'dataset_multilabel'
train_dir = f'{database_dir}/training/spectrogram'
val_dir = f'{database_dir}/val/spectrogram'
test_dir = f'{database_dir}/test/spectrogram'

# Directory where to store weights of the model and results
root_dir = "results"
# Create root directory for results if it does not exist
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

# Input dimension (number of subjects in our problem)
num_classes = 6

# Name of each gesture of the database
# CLASSES = [x for x in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, x))]
CLASSES = ['Alternative', 'Pop', 'Rock', 'Dance', 'Classical', 'Techno']
print(f'The classess to classify are: {CLASSES}')

# Parameters that characterise the spectrogram
img_height = 369
img_width = 496
img_channels = 3
color_mode = 'rgb'

The classess to classify are: ['Alternative', 'Pop', 'Rock', 'Dance', 'Classical', 'Techno']


### Configuration Training Parameters

In [3]:
# Parameters that configures the training process
batch_size = 32  # Batch size
epochs = 5  # Number of epochs
initial_lr = 1e-2   # Learning rate
seed = 42  # Random number
modelCNN = 'LSTM'  # RNN model which will be used
version = f'BS{batch_size}_E{epochs}_LR{initial_lr}'
experiment_dir = f'{root_dir}/{modelCNN}'

# Create experiment directory if it does not exist
if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

# Set random seed
if random_seed:
    seed = np.random.randint(0,2*31-1)
else:
    seed = 5
np.random.seed(seed)
tf.random.set_seed(seed)

### Loading of training, validation and test datasets of images
1. Dataframes with multiabel info
2. Training dataset
3. Validation dataset
4. Test dataset

In [4]:
# mlb = MultiLabelBinarizer()

# Training labels
train_df = pd.read_csv(f'{database_dir}/training_multilabel.csv', delimiter='\t')
train_df['Genres']= train_df['Genres'].apply(lambda x:x.split(", "))
# train_df['Genres'] = mlb.fit_transform(train_df['Genres'])
train_df['TRACK_ID'] = train_df['TRACK_ID'].astype(str)

# Validation labels
val_df = pd.read_csv(f'{database_dir}/validation_multilabel.csv', delimiter='\t')
val_df['Genres']= val_df['Genres'].apply(lambda x:x.split(", "))
val_df['TRACK_ID'] = val_df['TRACK_ID'].astype(str)

# Test labels
test_df = pd.read_csv(f'{database_dir}/test_multilabel.csv', delimiter='\t')
test_df['Genres'] = test_df['Genres'].apply(lambda x:x.split(", "))
test_df['TRACK_ID'] = test_df['TRACK_ID'].astype(str)

In [5]:
train_generator=ImageDataGenerator().flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col='TRACK_ID',
    y_col='Genres',
    batch_size=batch_size,
    seed=seed,
    shuffle=True,
    class_mode='categorical',
    classes=CLASSES,
    target_size=(100,100))

val_generator=ImageDataGenerator().flow_from_dataframe(
    dataframe=val_df,
    directory=val_dir,
    x_col='TRACK_ID',
    y_col='Genres',
    batch_size=batch_size,
    seed=seed,
    shuffle=True,
    class_mode='categorical',
    classes=CLASSES,
    target_size=(100,100))

test_generator=ImageDataGenerator().flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='TRACK_ID',
    y_col='Genres',
    batch_size=batch_size,
    seed=seed,
    shuffle=False,
    class_mode=None,
    target_size=(100,100))


# x_generator = ImageDataGenerator().flow_from_dataframe(
#     dataframe=train_df.tail(10),  # Seleccionar las últimas 10 filas del DataFrame
#     directory=train_dir,
#     x_col='TRACK_ID',
#     y_col='Genres',
#     batch_size=batch_size,
#     seed=seed,
#     shuffle=True,
#     class_mode='categorical',
#     classes=CLASSES
# )
# 
# # Obtener una muestra de lotes del generador de entrenamiento
# sample_batch = next(x_generator)
# 
# # Imprimir información sobre las columnas X e y del lote de muestra
# print("\nLote de muestra:")
# print("Columna X (features):")
# print(sample_batch[0])  # Este será el lote de imágenes (X)
# print("\nColumna y (etiquetas):")
# print(sample_batch[1])  # Este será el lote de etiquetas (y)

Found 1186 validated image filenames belonging to 6 classes.
Found 204 validated image filenames belonging to 6 classes.
Found 204 validated image filenames.


/Users/agv/opt/anaconda3/envs/DLMT/lib/python3.8/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 14 invalid image filename(s) in x_col="TRACK_ID". These filename(s) will be ignored.
  warnings.warn(


In [6]:
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,i] for i in range(6)])

## Training process
#### Available Models: CNN

### VGG-16

In [7]:
def build_model():
    model = Sequential()
    model.add(Conv2D(256, (3, 3), padding='same', input_shape=(100,100,img_channels)))
    model.add(Activation('relu'))
    model.add(Conv2D(256, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='sigmoid'))
    return model

## Model execution

In [8]:
#Model
model = build_model()
# Print the architecture of the model
model.summary()

2024-01-14 20:22:38.476095: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 256)     7168      
                                                                 
 activation (Activation)     (None, 100, 100, 256)     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 98, 98, 256)       590080    
                                                                 
 activation_1 (Activation)   (None, 98, 98, 256)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 256)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 49, 49, 256)       0         
                                                        

## Set model training process
#### Configuration of several training decisions:
1. Optimizer using `Adam`
2. Model training configuration using `compile` with `binary_crossentropy` due to the classification labeling

In [9]:
# 1. Configure optimizer
adam = Adam(learning_rate=initial_lr)

# 2. Configure training process
model.compile(loss = ['binary_crossentropy'],optimizer=adam, metrics=['accuracy'])

## Train the model
1. Load parameters from previous trainings if they exist.
2. Fit the model
3. Save the weights

In [ ]:
# Load pretrained model
weights_path = f"weights_{version}.h5" # Name of the file to store the weights
weights_file = Path(weights_path)
weights_load_path = f'{experiment_dir}/{weights_path}'
if weights_load_path:
    try:
        model.load_weights(weights_load_path)
        print("Loaded model from {}".format(weights_load_path))
    except:
        print("Impossible to find weight path. Returning untrained model")

# Fit the model
history = model.fit(train_generator, validation_data=val_generator, epochs=epochs, batch_size=batch_size)

# Save weights
weights_save_path = os.path.join(experiment_dir, weights_path)
model.save_weights(weights_save_path)

Impossible to find weight path. Returning untrained model
Epoch 1/5
 6/38 [===>..........................] - ETA: 4:43 - loss: 365145.6250 - accuracy: 0.1458

## Training Results
Accuracy and Loss obtained along the training process

In [ ]:
# plots.accloss(history, modelCNN, experiment_dir, version)
plt.figure()
plt.plot(history.history['dense_23_accuracy'])
plt.plot(history.history['val_dense_23_accuracy'])
plt.title(f'CNN Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')

# Save the figure
fig_save_path = f'{experiment_dir}/accuracy_{version}.png'
plt.savefig(fig_save_path)

# Show figure
plt.show()

# 2. Plot loss
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title(f'CNN Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper left')
fig_save_path = f'{experiment_dir}/loss_{version}.png'
plt.savefig(fig_save_path)
plt.show()

## Testing
### Model Testing
1. Compute the loss function and accuracy for the test data
2. Confusion Matrix obtained from testing results

In [ ]:
test_generator.reset()
pred=model.predict(test_generator, verbose=1)

In [ ]:
pred_bool = (pred >0.5)
predictions = pred_bool.astype(int)
columns=CLASSES
#columns should be the same order of y_col
results=pd.DataFrame(predictions, columns=columns)
results["Filenames"]=test_generator.filenames
ordered_cols=["Filenames"]+columns
results=results[ordered_cols]#To get the same column order
results.to_csv(f"{experiment_dir}/results_{version}.csv",index=False)

In [ ]:
# Evaluate model
scores = model.evaluate(test_generator, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
print("Loss: %.2f" % scores[0])

# Obtain results to present the confusion matrix
prob_class = model.predict(test_generator, batch_size=batch_size)
# Classified labels
y_pred = tf.argmax(prob_class, axis=-1)
# Ground truth
y_true = tf.argmax(tf.concat([label for image, label in test_generator], axis=0), axis=1)
# Visualize confusion matrix                                           
plots.cm(y_true, y_pred, modelCNN, CLASSES, experiment_dir, version)